# Review of MongoDB/Problems with 8.2 (courtesy of Andrew Leavitt)

In [1]:
# There are problems with file naming here
# python DB_from_file.py bball bbcoll bball_mm2.txt

import pymongo
 
client=pymongo.MongoClient('localhost', 27017)

# Do we have the bball database available?
client.list_database_names()

['bball', 'local']

In [2]:
db=client.bball
db.list_collection_names()

coll=db.bbcoll

#turning the cursor into a list of tweets so I can use it within python
docs=coll.find()

#docs

In [3]:
doclist=[tweet for tweet in docs]

doclist[1:10]

[{'_id': ObjectId('5d6125f263e2ac6c75f7f161'),
  'user': {'profile_background_tile': True,
   'friends_count': 468,
   'profile_sidebar_fill_color': '000000',
   'id_str': '55079974',
   'is_translation_enabled': False,
   'profile_link_color': '66A459',
   'followers_count': 386,
   'location': 'Cumberland',
   'protected': False,
   'default_profile_image': False,
   'contributors_enabled': False,
   'favourites_count': 3481,
   'profile_background_color': '131516',
   'statuses_count': 4629,
   'id': 55079974,
   'profile_banner_url': 'https://pbs.twimg.com/profile_banners/55079974/1489556560',
   'created_at': 'Thu Jul 09 00:13:28 +0000 2009',
   'profile_image_url_https': 'https://pbs.twimg.com/profile_images/811303542719967233/weEuBFS0_normal.jpg',
   'time_zone': 'Central Time (US & Canada)',
   'follow_request_sent': None,
   'listed_count': 2,
   'utc_offset': -18000,
   'lang': 'en',
   'is_translator': False,
   'name': 'Kahlen Donatell',
   'description': 'you doubleyou sto

In [4]:
# Was everything captured?
len(doclist)

4000

In [5]:
#printing out the head so I can see names of fields
print(doclist[:1])

[{'_id': ObjectId('5d6125f263e2ac6c75f7f160'), 'user': {'profile_background_tile': True, 'friends_count': 217, 'profile_sidebar_fill_color': 'EFEFEF', 'id_str': '513196438', 'is_translation_enabled': False, 'profile_link_color': 'F70808', 'followers_count': 256, 'location': '', 'protected': False, 'default_profile_image': False, 'contributors_enabled': False, 'favourites_count': 2187, 'profile_background_color': 'BFD0D9', 'statuses_count': 1104, 'id': 513196438, 'profile_banner_url': 'https://pbs.twimg.com/profile_banners/513196438/1437359097', 'created_at': 'Sat Mar 03 13:54:37 +0000 2012', 'profile_image_url_https': 'https://pbs.twimg.com/profile_images/732514428051181568/OkOVa8Ia_normal.jpg', 'time_zone': 'Eastern Time (US & Canada)', 'follow_request_sent': None, 'listed_count': 1, 'utc_offset': -14400, 'lang': 'en', 'is_translator': False, 'name': 'Will', 'description': "Do not worry about tomorrow, for tomorrow will worry about itself.-Matthew 6:34 HSC'20", 'profile_use_background

In [6]:
#Making a function to print a few tweets in a formatted manner
def print_tweet_data(tweets):
    for tweet in tweets:
        print('\nDate:', tweet['created_at'])
        print('From:', tweet['user']['name'])
        print('Message', tweet['text'])
        print('Friends', tweet['user']['friends_count'])

In [7]:
print_tweet_data(doclist[:5])


Date: Mon Mar 27 22:30:30 +0000 2017
From: Will
Message RT @HowardWKYT: The final seconds of the Kentucky-North Carolina game was an emotional roller coaster for Big Blue fans. #marchmadness #WKY…
Friends 217

Date: Mon Mar 27 22:30:22 +0000 2017
From: Kahlen Donatell
Message RT @WhistleSports: When you perfectly time the #UNC buzzer beater 😱🏀 #MarchMadness

(via:@SamuelGrubbs1 ) https://t.co/Ol2ibpZjB4
Friends 468

Date: Mon Mar 27 22:30:21 +0000 2017
From: Jesús
Message RT @HowardWKYT: The final seconds of the Kentucky-North Carolina game was an emotional roller coaster for Big Blue fans. #marchmadness #WKY…
Friends 156

Date: Mon Mar 27 22:30:14 +0000 2017
From: plug,
Message RT @BleacherReport: And then there were four... #MarchMadness https://t.co/0MbxpgAuUC
Friends 55

Date: Mon Mar 27 22:30:11 +0000 2017
From: Joni Dickerson
Message RT @mycarolinastdnt: RT if you'll be cheering on @GamecockWBB and @dawnstaley tonight. Let's Go Gamecocks! #MarchMadness
Friends 381


In [8]:
#Making a function to print out only the screennames of the people who tweeted
def print_tweet_sns(tweets):
    for tweet in tweets:
        print('Screenname:', tweet['user']['name'])

In [9]:
print_tweet_sns(doclist[:5])

Screenname: Will
Screenname: Kahlen Donatell
Screenname: Jesús
Screenname: plug,
Screenname: Joni Dickerson


# Accessing the Twitter API



In [31]:
''' Provides function that connects to Twitter
    Usage is shown in main test program
'''

import tweepy

# put the authorization codes here from your Twitter developer application
CONSUMER_KEY = 'Bh3oMqFDeFyfm4pot5frQAdKG'
CONSUMER_SECRET = 'ytikIPuy867T2fcGlVFFhlwSJd8aSR2CK0k1Yv4jk37MZf3noH'
OAUTH_TOKEN = '983117165716033537-OSY0vH40bmrSsian7n2wWXJ4TSeyWwH'
OAUTH_SECRET = '8w17C7Dj2oqyKzYw0Uibhff8V3ufPOjOD9f9nPaFLpeTT'

# login to Twitter with ordinary rate limiting
def oauth_login():
  # get the authorization from Twitter and save in the Tweepy package
  auth = tweepy.OAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
  auth.set_access_token(OAUTH_TOKEN,OAUTH_SECRET)
  tweepy_api = tweepy.API(auth)

  # if a null api is returned, give error message
  if (not tweepy_api):
      print ("Problem Connecting to API with OAuth")

  # return the Twitter api object that allows access for the Tweepy api functions
  return tweepy_api

# login to Twitter with extended rate limiting
#  must be used with the Tweepy Cursor to wrap the search and enact the waits
def appauth_login():
  # get the authorization from Twitter and save in the Tweepy package
  auth = tweepy.AppAuthHandler(CONSUMER_KEY,CONSUMER_SECRET)
  # apparently no need to set the other access tokens
  tweepy_api = tweepy.API(auth, wait_on_rate_limit=True, wait_on_rate_limit_notify=True)

  # if a null api is returned, give error message
  if (not tweepy_api):
      print ("Problem Connecting to API with AppAuth")

  # return the Twitter api object that allows access for the Tweepy api functions
  return tweepy_api
    
# Test program to show how to connect
if __name__ == '__main__':
  tweepy_api = oauth_login()
  print ("Twitter OAuthorization: ", tweepy_api)
  tweepy_api = appauth_login()
  print ("Twitter AppAuthorization: ", tweepy_api)

Twitter OAuthorization:  <tweepy.api.API object at 0x7fc688d4bcc0>
Twitter AppAuthorization:  <tweepy.api.API object at 0x7fc6ba2ee710>


In [23]:
import tweepy

# Function to extract tweets 
def get_tweets(username): 
        CONSUMER_KEY = 'OUv3sUhiBcXQf9g91x9Ayii2O'
        CONSUMER_SECRET = 'zkuQj5HnaVsFpz679U0XF54fl9lrhJKrrpqM5KFn5GuRLqEAOQ'
        OAUTH_TOKEN = '18249358-boz39qBA8cytXPhZ3WleTkmvDBSnWBri2OXGONDwv'
        OAUTH_SECRET = '1PBMhhE5FrPZoKD6Mpph5rkQAjC28TgtMqPcem1auwN9f'
          
        # Authorization to consumer key and consumer secret 
        auth = tweepy.OAuthHandler(CONSUMER_KEY, CONSUMER_SECRET) 
  
        # Access to user's access key and access secret 
        auth.set_access_token(OAUTH_TOKEN, OAUTH_SECRET) 
  
        # Calling api 
        api = tweepy.API(auth) 
  
        # 200 tweets to be extracted 
        number_of_tweets=200
        tweets = api.user_timeline(screen_name=username) 
  
        # Empty Array 
        tmp=[]  
  
        # create array of tweet information: username,  
        # tweet id, date/time, text 
        tweets_for_csv = [tweet.text for tweet in tweets] # CSV file created  
        for j in tweets_for_csv: 
  
            # Appending tweets to the empty array tmp 
            tmp.append(j)  
  
        # Printing the tweets 
        return(tmp) 

In [24]:
tweetlist = get_tweets('@jimmyjohns')

In [26]:
for tweet in tweetlist:
    print(tweet)

@amothcage https://t.co/1Al5Dy0FQm
@GetAtMeBROnen I'm sorry if we messed up! Please send the facts to my team here: https://t.co/ijC7Dwd4Ug so they ca… https://t.co/ldQN0PvGfq
@TomMoco5 I'm sorry for the delay last night! https://t.co/2uqvGc5Fvm
@MissikRice My #6 veggie is still on the menu! If you order it LBI and add extra cheese and avo, you end up with the old #13!
@Seth_Kloss https://t.co/mKjBQZJu1t
@tessallium Sounds like a solid Friday night to me!
@SeanJ0917 I'm sorry if we let you down! That's never my intention! Please send the facts to my team here:… https://t.co/gTuDX9ke1z
@haileyadmire Please send the facts to my team here: https://t.co/ijC7Dwd4Ug so they can educate the store and follow up with you directly!
@imJurld Thinking I like the way you think!
@thisjimbird I'm sorry ya don't dig my take in the famous Cuban!
@Stand4Truth1776 Please see https://t.co/IZZ7cWDFtF for full details!
@kylecain4311 I'm sorry, it's hard to tell from your photo, was something wrong?
@JENAHNJ

In [ ]:
print('Message', tweet['text'])

# Breakout group exercise

Using whatever search function that you like, collect 20 tweets that discuss the economic impact of coronavirus that had a low number of retweets <10 and 20 tweets that had a high number of retweets >100. We will report these tweets after breakout groups are over.